# 新手任务——故事上下句生成

# 0.Intro

开个烂坑（雾
<br>
应该说是给acci_nlp的新手任务开个头，具体要求：
![note_1](pic/note_1.jpg)

因为用jupyter用的比较多，打算先在这里实现一下，再思考下怎么写成完整的项目

# 1.load data

In [3]:
import os,sys
import io
import numpy as np
import pickle
import matplotlib.pyplot as plt
import random
import torch
import re

In [35]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(device)

cuda


In [3]:
def list_all_files(rootdir):
    rst = []
    list = os.listdir(rootdir)
    for f in list:
        path = os.path.join(rootdir,f)
        if os.path.isdir(path):
            rst.extend(list_all_files(path))
        if os.path.isfile(path):
            rst.append(path)
    return rst

In [47]:
o_data = 'origin_data/'
n_data = 'data/'

In [ ]:
o_data_list = [x for x in list_all_files(o_data) if os.path.splitext(x)[1] == '.txt']
o_data_list[:5]

In [5]:
print("txt文件总数：",len(o_data_list))

txt文件总数： 27955


In [12]:
with open('tmp/o_data_list.pickle','wb') as f_pickle:
        pickle.dump(o_data_list,f_pickle)

In [4]:
with open('tmp/o_data_list.pickle', 'rb') as f_pickle:
    o_data_list = pickle.load(f_pickle)

In [6]:
o_data_list[:5]

['origin_data/result_ZS\\七故事网\\古代故事\\“鸳鸯成双不分离”的故事.txt',
 'origin_data/result_ZS\\七故事网\\古代故事\\丁知县审鹅.txt',
 'origin_data/result_ZS\\七故事网\\古代故事\\不肯投胎的母亲.txt',
 'origin_data/result_ZS\\七故事网\\古代故事\\中药“王不留行”的故事.txt',
 'origin_data/result_ZS\\七故事网\\古代故事\\乾隆剃头.txt']

In [8]:
data = []

#### 合并句子集，得到初步的数据集
**简单地忽略了所有编码有问题的文件**

In [37]:
for df_path in o_data_list:
    with open(df_path,'r',encoding = 'utf8') as df:
        try:
            lines = df.readlines()
            for line in lines:
                line = line.strip()# 取出开头和结尾的空白符
                if len(line) == 0:
                    continue
                count = 0
                for c in line:
                    if (c >= 'a' and c <= 'z') or (c >= 'A' and c <= 'Z'):
                        count += 1
                if count / len(line) < 0.1:
                    data.append(line)
        except Exception as e:
            print(df_path)
            print(e)

origin_data/story\一千零一夜\一个小行囊的故事.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\一个苦行者的遭遇.txt
'utf-8' codec can't decode byte 0xc9 in position 0: invalid continuation byte
origin_data/story\一千零一夜\一千零一夜的来历.txt
'utf-8' codec can't decode byte 0xcf in position 0: invalid continuation byte
origin_data/story\一千零一夜\三代国王传奇.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\三岁小孩治坏蛋.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\与蛇为生的人.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\两个国王.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\两个国王斗智斗勇.txt
'utf-8' codec can't decode byte 0xb5 in position 2: invalid start byte
origin_data/story\一千零一夜\两个国王的故事.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\

origin_data/story\一千零一夜\牧人和恶棍.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\牧羊人与美女.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\狐狸和乌鸦做朋友.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\狐狸和乌鸦的故事.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\狐狸和狼.txt
'utf-8' codec can't decode byte 0xd4 in position 0: invalid continuation byte
origin_data/story\一千零一夜\狐狸和狼的故事.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\狮子的脚印.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\猎人和狮子的故事.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\一千零一夜\猎犬的故事.txt
'utf-8' codec can't decode byte 0xd5 in position 0: invalid continuation byte
origin_data/story\一千零一夜\猫和乌鸦.t

origin_data/story\上下五千年\三个神蛋.txt
'utf-8' codec can't decode byte 0xce in position 0: invalid continuation byte
origin_data/story\上下五千年\三保太监下西洋.txt
'utf-8' codec can't decode byte 0xc3 in position 0: invalid continuation byte
origin_data/story\上下五千年\三家瓜分晋国.txt
'utf-8' codec can't decode byte 0xbe in position 0: invalid start byte
origin_data/story\上下五千年\三征噶尔丹.txt
'utf-8' codec can't decode byte 0xb6 in position 2: invalid start byte
origin_data/story\上下五千年\三思而后行的故事.txt
'utf-8' codec can't decode byte 0xb2 in position 0: invalid start byte
origin_data/story\上下五千年\三箭定天山.txt
'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
origin_data/story\上下五千年\三英战吕布.txt
'utf-8' codec can't decode byte 0xbb in position 0: invalid start byte
origin_data/story\上下五千年\三让徐州.txt
'utf-8' codec can't decode byte 0xc1 in position 0: invalid start byte
origin_data/story\上下五千年\东宫里的棋手.txt
'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
origin_data/story\上下五千

origin_data/story\上下五千年\朋党之争.txt
'utf-8' codec can't decode byte 0xd3 in position 0: invalid continuation byte
origin_data/story\上下五千年\朋党的争吵.txt
'utf-8' codec can't decode byte 0xd4 in position 0: invalid continuation byte
origin_data/story\上下五千年\望梅止渴的故事.txt
'utf-8' codec can't decode byte 0xb6 in position 0: invalid start byte
origin_data/story\上下五千年\木兰从军的故事.txt
'utf-8' codec can't decode byte 0xb1 in position 0: invalid start byte
origin_data/story\上下五千年\本书大事年表.txt
'utf-8' codec can't decode byte 0xc9 in position 0: invalid continuation byte
origin_data/story\上下五千年\朱元璋抗元起义.txt
'utf-8' codec can't decode byte 0xd4 in position 0: invalid continuation byte
origin_data/story\上下五千年\机智李晟.txt
'utf-8' codec can't decode byte 0xb9 in position 0: invalid start byte
origin_data/story\上下五千年\李世民取东都.txt
'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte
origin_data/story\上下五千年\李光弼大破史思明.txt
'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
origin_data

'utf-8' codec can't decode byte 0xd4 in position 0: invalid continuation byte
origin_data/story\上下五千年\颜真卿刚强不屈.txt
'utf-8' codec can't decode byte 0xbe in position 0: invalid start byte
origin_data/story\上下五千年\飞将军李广.txt
'utf-8' codec can't decode byte 0xb9 in position 0: invalid start byte
origin_data/story\上下五千年\马不入厩的故事.txt
'utf-8' codec can't decode byte 0xb6 in position 0: invalid start byte
origin_data/story\上下五千年\马嵬驿兵变.txt
'utf-8' codec can't decode byte 0xe4 in position 0: invalid continuation byte
origin_data/story\上下五千年\马拉松.txt
'utf-8' codec can't decode byte 0xa1 in position 0: invalid start byte
origin_data/story\上下五千年\马良白眉.txt
'utf-8' codec can't decode byte 0xc8 in position 0: invalid continuation byte
origin_data/story\上下五千年\马谡失街亭.txt
'utf-8' codec can't decode byte 0xd6 in position 0: invalid continuation byte
origin_data/story\上下五千年\马邑诱敌战.txt
'utf-8' codec can't decode byte 0xba in position 0: invalid start byte
origin_data/story\上下五千年\马陵书树.txt
'utf-8' codec can't decode 

...

origin_data/story\童话故事\青蛙王子.txt
'utf-8' codec can't decode byte 0xd4 in position 2: invalid continuation byte
origin_data/story\童话故事\青蛙王子的故事.txt
'utf-8' codec can't decode byte 0xc7 in position 2: invalid continuation byte
origin_data/story\童话故事\青蛙王子续.txt
'utf-8' codec can't decode byte 0xbb in position 2: invalid start byte
origin_data/story\童话故事\青蛙的一千四百个谎话.txt
'utf-8' codec can't decode byte 0xc2 in position 2: invalid continuation byte
origin_data/story\童话故事\青蛙的哭诉.txt
'utf-8' codec can't decode byte 0xd4 in position 2: invalid continuation byte
origin_data/story\童话故事\青蛙的泪水.txt
'utf-8' codec can't decode byte 0xb4 in position 2: invalid start byte
origin_data/story\童话故事\青蛙神.txt
'utf-8' codec can't decode byte 0xb4 in position 2: invalid start byte
origin_data/story\童话故事\青蛙索比的明星梦.txt
'utf-8' codec can't decode byte 0xc3 in position 4: invalid continuation byte
origin_data/story\童话故事\青蛙请快跑.txt
'utf-8' codec can't decode byte 0xd5 in position 2: invalid continuation byte
origin_data/sto

origin_data/story\童话故事\驴子和他的影子.txt
'utf-8' codec can't decode byte 0xb8 in position 4: invalid start byte
origin_data/story\童话故事\驴子和它的主人.txt
'utf-8' codec can't decode byte 0xb6 in position 4: invalid start byte
origin_data/story\童话故事\驴子爸爸.txt
'utf-8' codec can't decode byte 0xd7 in position 4: invalid continuation byte
origin_data/story\童话故事\驴家族.txt
'utf-8' codec can't decode byte 0xcb in position 0: invalid continuation byte
origin_data/story\童话故事\驴小弟变石头.txt
'utf-8' codec can't decode byte 0xb8 in position 4: invalid start byte
origin_data/story\童话故事\驴皮公主.txt
'utf-8' codec can't decode byte 0xb4 in position 0: invalid start byte
origin_data/story\童话故事\驴皮公主的故事.txt
'utf-8' codec can't decode byte 0xb4 in position 2: invalid start byte
origin_data/story\童话故事\驼背小龙虾.txt
'utf-8' codec can't decode byte 0xd4 in position 2: invalid continuation byte
origin_data/story\童话故事\骄做的公主.txt
'utf-8' codec can't decode byte 0xb4 in position 2: invalid start byte
origin_data/story\童话故事\骄傲的公主.txt
'utf-8'

origin_data/story\童话故事\魔鬼的假日.txt
'utf-8' codec can't decode byte 0xa1 in position 2: invalid start byte
origin_data/story\童话故事\魔鬼的故事.txt
'utf-8' codec can't decode byte 0xb4 in position 2: invalid start byte
origin_data/story\童话故事\魔鬼身上的屋子.txt
'utf-8' codec can't decode byte 0xc9 in position 4: invalid continuation byte
origin_data/story\童话故事\鱼儿的梦想.txt
'utf-8' codec can't decode byte 0xba in position 2: invalid start byte
origin_data/story\童话故事\鱼和乌龟打的赌.txt
'utf-8' codec can't decode byte 0xcf in position 2: invalid continuation byte
origin_data/story\童话故事\鱼和戒指.txt
'utf-8' codec can't decode byte 0xb4 in position 2: invalid start byte
origin_data/story\童话故事\鱼和猫.txt
'utf-8' codec can't decode byte 0xb4 in position 2: invalid start byte
origin_data/story\童话故事\鱼和鱼鹰.txt
'utf-8' codec can't decode byte 0xb8 in position 2: invalid start byte
origin_data/story\童话故事\鱼国国王.txt
'utf-8' codec can't decode byte 0xd3 in position 0: invalid continuation byte
origin_data/story\童话故事\鱼在水里是怎么样呼吸的.txt
'utf-

origin_data/story\童话故事\黑天鹅与小蛤蟆.txt
'utf-8' codec can't decode byte 0xd4 in position 2: invalid continuation byte
origin_data/story\童话故事\黑妮子.txt
'utf-8' codec can't decode byte 0xb4 in position 2: invalid start byte
origin_data/story\童话故事\黑山鸡捷灵蒂.txt
'utf-8' codec can't decode byte 0xc1 in position 4: invalid start byte
origin_data/story\童话故事\黑星星.txt
'utf-8' codec can't decode byte 0xca in position 2: invalid continuation byte
origin_data/story\童话故事\黑板与黑板擦和粉笔.txt
'utf-8' codec can't decode byte 0xcc in position 4: invalid continuation byte
origin_data/story\童话故事\黑母鸡白母鸡.txt
'utf-8' codec can't decode byte 0xa1 in position 2: invalid start byte
origin_data/story\童话故事\黑母鸡的处事哲理.txt
'utf-8' codec can't decode byte 0xa1 in position 2: invalid start byte
origin_data/story\童话故事\黑池塘的秘密.txt
'utf-8' codec can't decode byte 0xb8 in position 2: invalid start byte
origin_data/story\童话故事\黑熊和小老鼠.txt
'utf-8' codec can't decode byte 0xba in position 2: invalid start byte
origin_data/story\童话故事\黑熊换鸡蛋.txt
'

origin_data/story\郑渊洁童话\水晶兔.txt
'utf-8' codec can't decode byte 0xb5 in position 2: invalid start byte
origin_data/story\郑渊洁童话\活车.txt
'utf-8' codec can't decode byte 0xce in position 2: invalid continuation byte
origin_data/story\郑渊洁童话\炸弹礼花.txt
'utf-8' codec can't decode byte 0xb5 in position 0: invalid start byte
origin_data/story\郑渊洁童话\父与子.txt
'utf-8' codec can't decode byte 0xb8 in position 2: invalid start byte
origin_data/story\郑渊洁童话\牌王西洛.txt
'utf-8' codec can't decode byte 0xa3 in position 2: invalid start byte
origin_data/story\郑渊洁童话\牛王醉酒.txt
'utf-8' codec can't decode byte 0xc4 in position 4: invalid continuation byte
origin_data/story\郑渊洁童话\牛魔王新传.txt
'utf-8' codec can't decode byte 0xb5 in position 2: invalid start byte
origin_data/story\郑渊洁童话\特别邮票.txt
'utf-8' codec can't decode byte 0xb5 in position 0: invalid start byte
origin_data/story\郑渊洁童话\狗王梦游.txt
'utf-8' codec can't decode byte 0xb8 in position 2: invalid start byte
origin_data/story\郑渊洁童话\狼窝里安了窃听器.txt
'utf-8' codec ca

In [46]:
len(data)

941538

In [39]:
data[:10]

['相传祝英台是一个美丽又聪明的姑娘，她不但会绣花做针线活，还会读书习字。',
 '一天，她忽然想到外面求学去，但她又觉得自家是一个女子，单身只影地到外面求学，很有些不便，不如化装一个男子的好。',
 '于是她就穿上男人的服装，离开家乡到很远的地方去了。',
 '她到了那个地方，拜过“圣人”和老先生，便和那些同学厮见了。',
 '这些同学的当中，有一个名叫梁山伯的，在祝英台的眼里看来，他是一个极其可爱的少年，因为他不但相貌清秀，性情也温和，读书尤其用功。',
 '因此她和他的感情，一天天亲密起来。',
 '两人整日共案读书，联床睡觉，她虽然热烈地爱他，但防范却非常严密，常常把她的书箱安置在两个床头的中间，书箱上面，放着一盆水，并告诉梁山伯：“你睡觉的时候，要安分一点，如果在床上乱翻乱滚，把盆里的水弄泼了，我马上报告老先生，叫他责罚你！”',
 '梁山伯听了她的话，只好规规矩矩的动也不敢动。',
 '因此祝英台在梁山伯面前始终没有露半点痕迹！',
 '梁山伯对祝英台虽那样要好，却毫不疑心她是一个女人，因为他本有些书呆子气，容易被她蒙混的。']

In [178]:
random_begin = random.randint(0, len(data))
data[random_begin:random_begin + 10]

['江水天天上涨，对面的土堤矮了。',
 '江面时刻漂浮着巨大的冰排。',
 '冰排重叠堆砌，在江中形成一座座小冰山。',
 '它会猝然崩溃，带着毁灭性的冲击力，被湍急的江流疾推而去。',
 '一天傍晚，我和班长巡逻完，并肩往哨所走。',
 '这个季节，春天虽然到了，乌苏里江虽然开化了，但气候并未明显转暖。',
 '大地上的雪，白天融化，夜晚冻结。',
 '江边罩着一层滑溜溜的冰壳。',
 '一脚踩下，发出嘎吱嘎吱的碎裂声。',
 '风，还是挺硬挺刺骨的。']

In [214]:
with open('tmp/data.pickle','wb') as f_pickle:
        pickle.dump(data,f_pickle)

In [50]:
with open(n_data + 'merged_data.txt','w',encoding = 'utf8') as f:
    for line in data:
        f.write(line + '\n')

In [228]:
with open('tmp/data.pickle', 'rb') as f_pickle:
    data = pickle.load(f_pickle)

In [168]:
# 最开始的情况
for line in data:
    if ' 'in line:
        print(line)

白雪公主：“好主意，你再贴上个防伪标记，写上 白雪公主，这样王后就会确信这是原产于我身上的零部件了。”
牛郎：“对面的女孩看过来～～～看过来～～看过来～～～ 这里的帅哥很精彩，请不要假装不理不睬～～～”
圆圆和方方手挽着手，肩并着肩，共唱一曲：“难忘今宵，难忘今宵， 无论天涯与海角，神州万里同怀抱……”
棕榈 叶的小叶片一定要保持一定的角度，就象磨坊风车的长臂一般。
对当时还很年轻的我来说，这真是一次有趣的经历 。
袁鹰 原名田钟洛。
管桦 原名鲍化普。
柯岩 原名冯恺。
任溶溶 原名任以奇。
圣野 原名周大鹿。
望安 原名李望安。
杨啸 原名杨瑞增。
金波 原名王金波。
田地 原名吴南熏。
聪聪 原名郄聪聪。
捣蛋鬼的日记 万巴著 思闵译
年 意大利军队进入罗马
年 加尼诺出生
至 肖毛校对
达林先生总是说，“我应该负全部责任。这都是我乔治·达林干的。， 。”
“我要你到巢里来，”那鸟叫道，尽量说得慢些，清楚些，“那样，你就可以漂到岸上去……可是我太累了 ， 不能再靠拢你 ，你得想法自己游过来。”
我在每个储蓄罐里放进了 马克。
我 真的开始生气了。
我写道： 金先生给我讲解的内客，我很快就明白了。
上周挣了马克； 金先生表扬了我。
马塞尔忽然显出惊慌失措的样子： “发生什么事了？”
鲁迪克是在世界各地都没有销售代理的 公司的创始人，她在童年时期卖过化妆品。
× （三个百分点）利息如果你将资金借给银行（就是说把钱存人一个账户），银行就付给你一定的金额作为回报。
安房直子的主要作品有《被施了魔法的舌头》《风与树的歌》《手绢上的花田》《白鹦鹉的森林》《银孔雀》《紫丁香大街的帽子店》《黄昏海的故事》《天鹿》《遥远的野玫瑰村》《花香小镇》《冬吉和熊的故事》《山的童话：风的旱冰鞋》《狗尾草的原野豆腐店主的故事》《红玫瑰旅馆的客人》《直到花豆煮熟为止小夜的故事》等，其代表作是《手绢上的花田》《北风遗忘的手绢》《风和树的歌》《遥远的野蔷薇村》《山的童话·风溜旱冰》《谁也看不见的阳台》等 。
东街 三三十一
菊屋酒店 收
莱曼·弗兰克·鲍姆 著，陈伯吹 译
」，如 。
》 《周日的所罗门·格朗迪》
' ，
' ，
蛋也不能恢复原来的样子 。
， ，
， ，
， ， ， ，
这首童谣大约是世纪中叶在民间出现，但在更早些时候似乎就有了迹象，在莎士比亚的《第夜》中有这样的一句话： ，


In [184]:
symbols = ['，','。','”','“','！','!','：',':']
# symbols = ["'",';','"']
# symbols = ['(',')','<',">",'《',"》",'（',"）"]
for e1 in symbols:
    for e2 in symbols:
        st = e1+ e2
        try:
            pattern = re.compile(r'.*'+st)
            xx = (len(data))
            data = [line for line in data if re.match(pattern, line) == None]
            if xx != len(data):
                print(st)
                print(len(data))
        except:
            print(st)
    

，：
768493
，:
768492
”：
768452
“：
768440
！：
768438
：，
768429
：。
768404
：”
768299
：“
748852
：！
748850
：：
748845
：:
748816
:。
748800
:”
748797
:“
748462


In [202]:
print(len(data))
xx = [line for line in data if line != '/']
print(len(xx))
data = xx

748220
748220


# 2.charRNN
**数据集：data/merged_data.txt**

In [1]:
import torch as t
from torch import nn
import visdom
import torchvision
import numpy as np
import torch.nn.functional as F
import os
import sys
import re
import time
import random
USE_CUDA = t.cuda.is_available()
device = t.device("cuda" if USE_CUDA else "cpu")
print(device)

cuda


### 先进行了很多数据的操作，最后得到了一些文件
#### data/charrnn_data.txt 筛选后的文本元数据
#### tmp/charrnn_dataa.pickle 文本列表 list
#### tmp/charrnn_chars.pickle 获取到的word list和对应的频次 list
#### tmp/charrnn_dict.pickle 得到的字典 \[char2id,id2char\] 每个字典为dict


In [277]:
# # 进一步简化数据集，仅保留，。！：？的句子
# symbols = ['”','“',"'",';','"','(',')','<',">",'《',"》",'（',"）"]
# index = 0
# for line in data:
#     if '…' in line:
# #         print(line)
#         data.remove(line)
#         continue
# #     for e in symbols:
# #         if e in line:
# #             print(line)
# #             data.remove(line)
# #             break
#     index += 1
#     if index % 1000 == 0:
#         print("index :" , index, "data len:" ,len(data))

index : 1000 data len: 656686
index : 2000 data len: 656686
index : 3000 data len: 656686
index : 4000 data len: 656684
index : 5000 data len: 656684
index : 6000 data len: 656684
index : 7000 data len: 656684
index : 8000 data len: 656684
index : 9000 data len: 656684
index : 10000 data len: 656684
index : 11000 data len: 656684
index : 12000 data len: 656683
index : 13000 data len: 656683
index : 14000 data len: 656683
index : 15000 data len: 656683
index : 16000 data len: 656683
index : 17000 data len: 656683
index : 18000 data len: 656682
index : 19000 data len: 656682
index : 20000 data len: 656682
index : 21000 data len: 656682
index : 22000 data len: 656682
index : 23000 data len: 656682
index : 24000 data len: 656682
index : 25000 data len: 656682
index : 26000 data len: 656682
index : 27000 data len: 656682
index : 28000 data len: 656682
index : 29000 data len: 656682
index : 30000 data len: 656682
index : 31000 data len: 656682
index : 32000 data len: 656682
index : 33000 dat

index : 516000 data len: 652706
index : 517000 data len: 652706
index : 518000 data len: 652706
index : 519000 data len: 652698
index : 520000 data len: 652683
index : 521000 data len: 652673
index : 522000 data len: 652673
index : 523000 data len: 652673
index : 524000 data len: 652673
index : 525000 data len: 652673
index : 526000 data len: 652647
index : 527000 data len: 652645
index : 528000 data len: 652643
index : 529000 data len: 652636
index : 530000 data len: 652626
index : 531000 data len: 652617
index : 532000 data len: 652609
index : 533000 data len: 652609
index : 534000 data len: 652608
index : 535000 data len: 652608
index : 536000 data len: 652608
index : 537000 data len: 652599
index : 538000 data len: 652598
index : 539000 data len: 652595
index : 540000 data len: 652584
index : 541000 data len: 652572
index : 542000 data len: 652559
index : 543000 data len: 652552
index : 544000 data len: 652537
index : 545000 data len: 652521
index : 546000 data len: 652497
index : 

In [5]:
with open('tmp/charrnn_data.pickle', 'rb') as f_pickle:
    data = pickle.load(f_pickle)
len(data)

651763

In [7]:
# 建立字典映射
chars = {}
for line in data:
    for c in line:
        chars[c] = chars.get(c,0) + 1
chars = sorted(chars.items(), key = lambda x : x[1], reverse = True)
vocab_list = [char[0] for char in chars]
print("共%d个汉字" % len(vocab_list))


共6206个汉字


In [14]:
chars[:20]

[('，', 905315),
 ('的', 565013),
 ('。', 563071),
 ('一', 328500),
 ('了', 309974),
 ('他', 202302),
 ('是', 183746),
 ('不', 172770),
 ('在', 169885),
 ('我', 143517),
 ('来', 138563),
 ('上', 129948),
 ('着', 126166),
 ('子', 124776),
 ('有', 123313),
 ('这', 119103),
 ('到', 117652),
 ('个', 116007),
 ('地', 113411),
 ('人', 107336)]

In [15]:
with open('tmp/charrnn_chars.pickle','wb') as f:
    pickle.dump(chars,f)

In [9]:
char2id = {c : i for i,c in enumerate(vocab_list)}
id2char = {i : c for i,c in enumerate(vocab_list)}

# with open('tmp/charrnn_dict.pickle','wb') as f:
#     pickle.dump([char2id, id2char],f)

In [11]:
len(char2id)

6206

In [13]:
with open('data/charrnn_data.txt','w',encoding = 'utf8') as f:
    for line in data:
        f.write(line + '\n')

In [6]:
print(len(data))
random_begin = random.randint(0, len(data))
data[random_begin:random_begin + 10]

651763


['这会儿，米加坐在树底下，黑碳站在树上，红辣椒飞下来和米加说话，他们并排坐在野兔灰小姐种的两棵草旁边。',
 '黑碳就像是给他们站岗的警卫。',
 '米加的手里拿着一张设计图，这是他这两天刚刚画的。',
 '红辣椒说。',
 '红辣椒看不懂这样的图。',
 '乌鸦大部分会说话，不太动笔的。',
 '米加说。',
 '红辣椒说。',
 '米加卷起图纸。',
 '他想起他设计的洗衣机的图纸了。']

In [1]:
# 模型构建从这里开始
import torch as t
from torch.utils.data import DataLoader
from torch import nn
import visdom
import torchvision
import numpy as np
import torch.nn.functional as F
import os
import sys
import re
import time
import pickle
import random
USE_CUDA = t.cuda.is_available()
device = t.device("cuda" if USE_CUDA else "cpu")
print(device)

cuda


In [9]:
class Config(object):
    # 用来写训练参数
    use_gpu = True
    
    # 先是dataset中的参数
    data_path = 'data/charrnn_data.txt'
#     with open('tmp/charrnn_data.pickle','rb') as f:
#         data_list = pickle.load(f)
    with open('tmp/charrnn_dict.pickle','rb') as f:
        data_dict = pickle.load(f)
    char2id = data_dict[0]
    id2char = data_dict[1]
    n_step = 20
    
    
    # model中的参数
    vocab_size = len(char2id)
    embedding_dim = 128
    hidden_dim = 128
    num_layers = 2
    dropout = 0.5
    
    lr = 1e-3
    
    batch_size = 8
    
    epochs = 5
    
opt = Config()

In [3]:
def f_id2char(list,opt):
    rst = [opt.id2char[int(e)] for e in list]
    return rst

In [4]:
class CharRnnDataset(t.utils.data.Dataset):
    def __init__(self, opt):
        data_path = opt.data_path
        char2id = opt.char2id
        n_step = opt.n_step
        with open(data_path,'r',encoding = 'utf8') as f:
            data_list = f.readlines()
        data_list = [char2id[e] for line in data_list for e in line if e != '\n'] # 转为长列表，且汉字转数字
        num_seq = int(len(data_list) / n_step)# 限定每次喂入的序列长度（太长的话会梯度消失）
        # 存储数据矩阵的维度
        self.num_seq = num_seq
        self.n_step = n_step
        # 舍去最后几个不够一行的
        data_list = data_list[:num_seq * n_step]
        # 存成矩阵
        arr = np.array(data_list).reshape((num_seq,-1))
        self.arr = t.from_numpy(arr)
    def __getitem__(self, item):
        # 获取第item次的数据
        x = self.arr[item,:]
        # 将最后一个字符的输出 label 定为输入的第一个字符，也就是"床前明月光"的输出是"前明月光床"
        y = t.zeros(x.shape)
        y[:-1],y[-1] = x[1:],x[0]
        return x,y
    def __len__(self):
        return self.num_seq

In [5]:
# 试验一下Dataset
train_set = CharRnnDataset(opt)
x,y = train_set[0]

In [6]:
f_id2char(x,opt)

['相',
 '传',
 '祝',
 '英',
 '台',
 '是',
 '一',
 '个',
 '美',
 '丽',
 '又',
 '聪',
 '明',
 '的',
 '姑',
 '娘',
 '，',
 '她',
 '不',
 '但']

In [7]:
# 定义训练模型
class CharRnn(nn.Module):
    def __init__(self, opt):
        super(CharRnn, self).__init__()
        vocab_size = opt.vocab_size
        embedding_dim =  opt.embedding_dim
        hidden_dim = opt.hidden_dim
        num_layers = opt.num_layers
        dropout = opt.dropout
        
        self.use_gpu = opt.use_gpu
        self.hidden_dim = hidden_dim
        self.embedding_dim = opt.embedding_dim
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        # word embedding层
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # lstm rnn层
        self.rnn = nn.LSTM(embedding_dim, self.hidden_dim, num_layers, dropout)
        # 线性层
        self.linear = nn.Linear(self.hidden_dim, vocab_size)
    
    
    def forward(self, input, hidden = None):
        num_seq, n_step = input.size()
        if hidden is None:
            h_0 = input.data.new(self.num_layers, n_step, self.hidden_dim).fill_(0.)
            c_0 = input.data.new(self.num_layers, n_step, self.hidden_dim).fill_(0.)
            if self.use_gpu:
                h_0 = h_0.cuda()
                c_0 = c_0.cuda()
        else:
            h_0,c_0 = hidden
        # 第一层:embedding
        embeddings = self.embeddings(input)# (num_seq, n_step, embeddings_dim)
        
        # 第二层:lstm
        output2, hidden = self.lstm(embeddings, (h_0,c_0))#(num_seq, n_step, hidden_dim)
        
        # 第三层:linear
        output3 = self.linear(output2.view(num_seq * n_step, -1))#(num_seq * n_step, vocab_size)
        return output,hidden

In [8]:
train_data = DataLoader(train_set, opt.batch_size, True, num_workers=4)
model = CharRnn(opt)
if opt.use_gpu:
    model = model.cuda()

criterion = nn.CrossEntropyLoss()

optimizer = t.optim.Adam(model.parameters(), lr = opt.lr)

In [ ]:
for e in range(opt.epochs):
    train_loss = 0
    for data in train_data:
        x, y = data
        if  opt.use_gpu:
            x = x.cuda()
            y = y.cuda()
            
        score, _ = model(x)
        
        loss = criterion(score,y.view(-1))
        
        optimizer.zero_grad()
        loss.backward()
        
        nn.utils.clip_grad_norm(model.parameters(), 5)
        optimizer.step()
        
        train_loss += loss.data[0]
        
    print('epoch: {}, perplexity is: {:.3f}, lr:{:.1e}'.format(e+1, np.exp(train_loss / len(train_data)), optimizer.lr))